In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

from compute_ET_green import compute_et_green

from utils import ee_utils

import ee
import geemap

In [3]:
ee.Initialize(project="thurgau-irrigation")

## Define the region of interest

In [4]:
# For the canton of Thurgau
cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

## Importing all assets: land use maps, vegetation periods, WaPOR ET data.

In [5]:
years_to_process = range(2018, 2023) 

nutzung_collection = ee.FeatureCollection("projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area")

double_cropping_image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/vegetation_periods_2018-2023")


WaPOR_assets = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal"
WaPOR_et_collections = ee_utils.merge_collections(years=years_to_process, asset_name=WaPOR_assets)

## Filtering out potentially irrigated crops and refeence rainfed fields

In [6]:
# # Add double cropping information to the feature collection
# nutzung_collection_with_double_crop = add_double_cropping_info(
#     nutzung_collection, double_cropping_image_2018
# )

# not_irrigated_crops = get_crops_to_exclude()
# rainfed_crops = get_rainfed_reference_crops()

# exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

# potentially_rainfed, rainfed_fields = filter_crops(
#     nutzung_collection_with_double_crop, exclude_filter, rainfed_filter
# )

# print("Filtered fields count:", potentially_rainfed.size().getInfo())
# print("Rainfed reference fields count:", rainfed_fields.size().getInfo())

Filtered fields count: 12931
Rainfed reference fields count: 21958


## Computing ET green and exporting the results

In [7]:
# et_image = WaPOR_et_collections.first()

# et_green_result = compute_et_green(et_image, rainfed_fields)

In [6]:
def export_image_to_asset(
    image: ee.Image,
    asset_id: str,
    task_name: str,
    year: int,
    aoi: ee.Geometry,
    max_pixels: int = 1e13,
) -> ee.batch.Task:
    """
    Export an image to an Earth Engine asset.
    """
    task = ee.batch.Export.image.toAsset(
        image=image,
        description=task_name,
        assetId=asset_id,
        region=aoi,
        scale=10,
        maxPixels=max_pixels,
    )
    print(f"Exporting {task_name} for {year} to {asset_id}")
    task.start()
    return task


double_cropping_list = double_cropping_image_collection.toList(
    double_cropping_image_collection.size()
)
WaPOR_et_collections_list = WaPOR_et_collections.toList(WaPOR_et_collections.size())

not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

tasks = []

total_tasks = 0

for year in years_to_process:
    j = year - 2018
    nutzung_collection_year = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
    )
    double_cropping_image_year = ee.Image(double_cropping_list.get(j))

    nutzung_collection_with_double_crop_year = add_double_cropping_info(
        nutzung_collection_year, double_cropping_image_year
    )

    potentially_rainfed_year, rainfed_fields_year = filter_crops(
        nutzung_collection_with_double_crop_year, exclude_filter, rainfed_filter
    )

    year_tasks = 0

    for i in range(36):
        et_image_year = ee.Image(WaPOR_et_collections_list.get(j * 36 + i))
        et_green_result_year = compute_et_green(et_image_year, rainfed_fields_year)

        dekadal = i % 3 + 1
        month = i // 3 + 1

        task_name = f"ET_green_{year}-{month:02d}_D{dekadal}"
        asset_id = (
            f"projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/{task_name}"
        )

        task = export_image_to_asset(
            et_green_result_year, asset_id, task_name, year, aoi_buffered
        )
        tasks.append(task)
        year_tasks += 1
        total_tasks += 1

    print(f"Year {year} processing complete. Started {year_tasks} tasks.")

print(f"All processing complete. Total tasks started: {total_tasks}")

Exporting ET_green_2018-01_D1 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-01_D1
Exporting ET_green_2018-01_D2 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-01_D2
Exporting ET_green_2018-01_D3 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-01_D3
Exporting ET_green_2018-02_D1 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-02_D1
Exporting ET_green_2018-02_D2 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-02_D2
Exporting ET_green_2018-02_D3 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-02_D3
Exporting ET_green_2018-03_D1 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-03_D1
Exporting ET_green_2018-03_D2 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/ET_green_2018-03_D2
Exportin

## Testing that it all went fine

In [7]:
et_green_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022")